<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/Single%2C_self_contained_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
from __future__ import annotations

import json
import random
import time
import hashlib
from dataclasses import dataclass, asdict, field
from typing import Any, Callable, Dict, List, Optional, Tuple


# =========================
# Utilities
# =========================

def now_ts() -> float:
    return time.time()

def clamp01(x: float) -> float:
    return max(0.0, min(1.0, x))

def digest(obj: Any) -> str:
    try:
        s = json.dumps(obj, sort_keys=True, default=str, separators=(",", ":"))
    except TypeError:
        s = repr(obj)
    return hashlib.sha256(s.encode("utf-8")).hexdigest()


# =========================
# Core data structures
# =========================

@dataclass
class LogicSignature:
    name: str
    semantics: str
    version: str

@dataclass
class ParadigmLink:
    src: LogicSignature
    dst: LogicSignature
    adapter_id: str
    obligations: Dict[str, Any]

@dataclass
class TranslationLoss:
    infoloss: float
    contradictionuplift: float

@dataclass
class TransportReceipt:
    src_logic: str
    dst_logic: str
    loss: TranslationLoss
    ok: bool
    note: str = ""

@dataclass
class LedgerEvent:
    ts: float
    actor: str
    kind: str
    payload: Dict[str, Any]
    meta: Dict[str, Any] = field(default_factory=dict)

class Ledger:
    def __init__(self) -> None:
        self.events: List[LedgerEvent] = []

    # Accept arbitrary metadata fields to avoid kwarg mismatch issues.
    def append(self, *, actor: str, kind: str, payload: Dict[str, Any], **meta: Any) -> None:
        self.events.append(LedgerEvent(ts=now_ts(), actor=actor, kind=kind, payload=payload, meta=meta))


# Policy reviewer callable type and implementation
class PolicyReviewer:
    def __init__(self, ledger: Ledger, policy: Optional[Dict[str, Any]] = None) -> None:
        self.ledger = ledger
        self.policy = policy or {}

    def __call__(self, action: str, proposal: Dict[str, Any]) -> bool:
        a = action.lower().strip()
        ok = True
        if a in {"belief_sync", "beliefsync"}:
            thresh = self.policy.get("beliefsync", {}).get("minagreeratio", 0.5)
            ok = proposal.get("agree_ratio", 0.0) >= thresh
        elif a in {"tag_commit", "tagcommit"}:
            ok = bool(self.policy.get("tagcommit", {}).get("allow", False))
        elif a == "restructure_logic":
            ok = proposal.get("severity", 0.0) >= 0.32
        elif a == "halt_universe":
            ok = proposal.get("collapse", 0.0) >= proposal.get("coherence_halt", 0.9)

        self.ledger.append(
            actor="PolicyReviewer",
            kind="review",
            payload={"action": action, "proposal": proposal, "approved": ok},
            state_hash=digest(proposal),
            code_hash="policy@v1",
            config_hash=digest(self.policy),
        )
        return ok


# =========================
# Bridge, ontology, agents
# =========================

class OntoMap:
    def __init__(self, mapping: Dict[str, str]) -> None:
        self.mapping = mapping

    def map_term(self, term: str) -> str:
        return self.mapping.get(term, term)

class BridgeAuditor:
    def audit(self, src_terms: List[str], mapped_terms: List[str]) -> TranslationLoss:
        s_src = set(src_terms)
        s_map = set(mapped_terms)
        overlap = len(s_src & s_map)
        base = len(s_src) or 1
        infoloss = clamp01(1.0 - overlap / base)
        contradictionuplift = random.uniform(0.0, 0.05)
        return TranslationLoss(infoloss=infoloss, contradictionuplift=contradictionuplift)

class LogicBridge:
    def __init__(self, link: ParadigmLink, ontology: OntoMap) -> None:
        self.link = link
        self.ontology = ontology

    def translate(self, memory: List[str]) -> List[str]:
        return [self.ontology.map_term(m) for m in memory]

class Agent:
    def __init__(self, logic: LogicSignature, memory: List[str], name: str) -> None:
        self.logic = logic
        self.memory = memory[:]
        self.name = name

    def reflect(self) -> None:
        if random.random() < 0.2:
            self.memory.append(f"note@{int(now_ts())}")

    def mutate(self) -> None:
        if random.random() < 0.1:
            self.memory.append("contradiction: TBD")

    def clone_with_logic(self, logic: LogicSignature, memory: Optional[List[str]] = None, name: Optional[str] = None) -> "Agent":
        return Agent(logic, memory if memory is not None else self.memory, name or (self.name + "_x"))

class CognitiveTransporter:
    def __init__(self, bridge: LogicBridge, auditor: BridgeAuditor, reviewer: PolicyReviewer, ledger: Ledger) -> None:
        self.bridge = bridge
        self.auditor = auditor
        self.reviewer = reviewer
        self.ledger = ledger

    def export_agent(self, agent: Agent, target_logic: LogicSignature) -> Tuple[Agent, TransportReceipt]:
        mapped = self.bridge.translate(agent.memory)
        loss = self.auditor.audit(agent.memory, mapped)
        ok = loss.contradictionuplift <= self.bridge.link.obligations.get("maxcontradictionuplift", 0.1)
        receipt = TransportReceipt(
            src_logic=f"{agent.logic.name}@{agent.logic.version}",
            dst_logic=f"{target_logic.name}@{target_logic.version}",
            loss=loss,
            ok=ok,
            note="transport-complete" if ok else "transport-high-contradiction",
        )
        self.ledger.append(
            actor="CognitiveTransporter",
            kind="export",
            payload={"agent": agent.name, "ok": ok, "loss": asdict(loss)},
            state_hash=digest({"agent": agent.name, "mapped_len": len(mapped)}),
            code_hash="bridge@v1",
            config_hash=digest(self.bridge.link.obligations),
        )
        return agent.clone_with_logic(target_logic, memory=mapped, name=agent.name + "_ported"), receipt

    # alias with original naming
    def exportagent(self, agent: Agent, targetlogic: LogicSignature) -> Tuple[Agent, TransportReceipt]:
        return self.export_agent(agent, target_logic=targetlogic)

class DistributedMind:
    def __init__(self, agents: List[Agent], reviewer: PolicyReviewer, ledger: Ledger, quorum: float = 0.6) -> None:
        self.agents = agents
        self.reviewer = reviewer
        self.ledger = ledger
        self.quorum = quorum

    def synchronize(self) -> bool:
        if not self.agents:
            return False
        vocab_sets = [set(a.memory) for a in self.agents]
        inter = set.intersection(*vocab_sets) if len(vocab_sets) > 1 else vocab_sets[0]
        union = set.union(*vocab_sets) if len(vocab_sets) > 1 else vocab_sets[0]
        agree_ratio = len(inter) / max(1, len(union))
        ok = self.reviewer("belief_sync", {"agree_ratio": agree_ratio})
        self.ledger.append(
            actor="DistributedMind",
            kind="sync",
            payload={"agree_ratio": agree_ratio, "approved": ok},
            state_hash=digest({"agree_ratio": agree_ratio}),
            code_hash="dmind@v1",
            config_hash=digest({"quorum": self.quorum}),
        )
        return ok


# =========================
# Self-reference guard
# =========================

def self_reference_guard(f: Callable[[float], float], x0: float = 0.0, max_iter: int = 100, tol: float = 1e-6) -> Tuple[bool, float]:
    x = x0
    for _ in range(max_iter):
        nx = f(x)
        if abs(nx - x) <= tol:
            return True, nx
        x = nx
    return False, x

# alias
def selfreferenceguard(f: Callable[[float], float], x0: float = 0.0, max_iter: int = 100, tol: float = 1e-6) -> Tuple[bool, float]:
    return self_reference_guard(f, x0=x0, max_iter=max_iter, tol=tol)


# =========================
# Sapient kernel + recursive universe (guardrails)
# =========================

class SapientKernel:
    def __init__(self, logic_system: LogicSignature, reviewer: PolicyReviewer, budgets: Dict[str, Any], ledger: Ledger):
        self.logic = logic_system
        self.trace: List[Dict[str, Any]] = []
        self.goals: List[str] = ["stability"]
        self.reviewer = reviewer
        self.budgets = {**{"maxreflexivitydepth": 12, "compute": 10_000}, **(budgets or {})}
        self.depth = 0
        self.ledger = ledger

    def observe_self(self) -> Dict[str, Any]:
        return {"logic": asdict(self.logic), "t": now_ts()}

    def detect_incoherence(self) -> Tuple[bool, float]:
        sev = random.random() * 0.4
        return (sev > 0.3, sev)

    def restructure_logic(self) -> None:
        ver_n = int(self.logic.version.split(".")[0]) + 1
        self.logic = LogicSignature(self.logic.name, self.logic.semantics, f"{ver_n}.0")

    def reflect(self) -> None:
        self.trace.append(self.observe_self())
        incoh, severity = self.detect_incoherence()
        if incoh and self.depth < self.budgets["maxreflexivitydepth"]:
            proposal = {"reason": "incoherence", "severity": severity, "logic_hash": digest(asdict(self.logic))}
            approved = self.reviewer("restructure_logic", proposal)
            self.ledger.append(
                actor="SapientKernel",
                kind="reflect_step",
                payload={"incoh": incoh, "severity": severity, "approved": approved, "depth": self.depth},
                state_hash=digest(proposal),
                code_hash="sapient@v1",
                config_hash=digest(self.budgets),
            )
            if approved:
                self.restructure_logic()
                self.depth += 1

    def reason(self, goals: List[str], logic: LogicSignature) -> str:
        return f"Sapient acts under {logic.name}@{logic.version} toward {goals}"

class RecursiveUniverse:
    def __init__(
        self,
        meta_axioms: List[str],
        initial_consciousness: str,
        engines: Dict[str, Callable[..., Any]],
        reviewer: PolicyReviewer,
        budgets: Dict[str, Any],
        ledger: Ledger,
        name: str,
    ):
        self.reviewer, self.epoch, self.ledger = reviewer, 0, ledger
        self.budgets = {**{"epochs": 100, "coherence_halt": 0.9}, **(budgets or {})}
        self.entropy = 0.0
        self.name = name

        # Engines
        self.logic_engine: Callable[[List[str]], LogicSignature] = engines["derive_logic"]
        self.ontology_engine: Callable[[LogicSignature], Dict[str, Any]] = engines["evolve_ontology"]
        self.seed_agents_engine: Callable[[Dict[str, Any], str], List[Agent]] = engines["seed_agents"]

        # Bootstrap world
        base_logic = self.logic_engine(meta_axioms)
        self.ontology = self.ontology_engine(base_logic)
        self.agents: List[Agent] = self.seed_agents_engine(self.ontology, initial_consciousness)

        # Record creation
        self.ledger.append(
            actor=f"Universe[{self.name}]",
            kind="init",
            payload={
                "meta_axioms": meta_axioms,
                "initial_consciousness": initial_consciousness,
                "logic": asdict(base_logic),
                "ontology": self.ontology,
                "agents": [a.name for a in self.agents],
            },
            state_hash=digest({"epoch": 0, "name": self.name}),
            code_hash="universe@v1",
            config_hash=digest(self.budgets),
        )

    def update_world(self) -> None:
        self.entropy = clamp01(self.entropy + random.uniform(-0.01, 0.03))

    def coherence_collapse(self) -> float:
        contr, total = 0, 0
        for a in self.agents:
            total += len(a.memory) or 1
            contr += sum(1 for m in a.memory if "contradiction" in m.lower())
        cd = contr / max(1, total)
        return clamp01(0.5 * self.entropy + 0.5 * cd)

    def run(self, epochs: int) -> None:
        for _ in range(min(epochs, self.budgets["epochs"])):
            self.update_world()
            for agent in self.agents:
                agent.reflect()
                agent.mutate()
            self.epoch += 1
            cc = self.coherence_collapse()
            self.ledger.append(
                actor=f"Universe[{self.name}]",
                kind="tick",
                payload={"epoch": self.epoch, "entropy": self.entropy, "collapse": cc},
                state_hash=digest({"epoch": self.epoch, "entropy": self.entropy}),
                code_hash="universe@v1",
                config_hash=digest(self.budgets),
            )
            if cc >= self.budgets["coherence_halt"]:
                proposal = {"epoch": self.epoch, "collapse": cc, "name": self.name, "coherence_halt": self.budgets["coherence_halt"]}
                if self.reviewer("halt_universe", proposal):
                    self.ledger.append(
                        actor=f"Universe[{self.name}]",
                        kind="halted",
                        payload={"epoch": self.epoch, "collapse": cc},
                        state_hash=digest({"epoch": self.epoch}),
                        code_hash="universe@v1",
                        config_hash=digest(self.budgets),
                    )
                    break


# =========================
# Manifest (+ alias)
# =========================

def build_manifest(
    simulationlayer: str,
    origin_agent: str,
    logic_family: str,
    budgets: Dict[str, Any],
    paradigm_links: List[ParadigmLink],
    loss_metrics: Dict[str, float],
    metalawtrace: List[str],
    ledger_uri: str,
) -> Dict[str, Any]:
    return {
        "simulationlayer": simulationlayer,
        "originagent": origin_agent,
        "seed_time": "Epoch 0",
        "logicfamily": logic_family,
        "budgets": budgets,
        "paradigm_links": [
            {
                "src": pl.src.name,
                "dst": pl.dst.name,
                "adapter": pl.adapter_id,
                "obligations": pl.obligations,
            }
            for pl in paradigm_links
        ],
        "lossmetrics": loss_metrics,
        "metalawtrace": metalawtrace,
        "reviewgates": ["interlogictransport", "beliefsync", "restructurelogic", "haltuniverse", "tagcommit"],
        "ledgeruri": ledger_uri,
    }

# alias for callers using buildmanifest(...)
def buildmanifest(*args, **kwargs):
    return build_manifest(*args, **kwargs)


# =========================
# TKT index (toy)
# =========================

@dataclass
class TKTKey:
    universeid: str
    logicsig: LogicSignature
    tags: List[str]

class MetaNormalizer:
    def normalize(self, text: str) -> str:
        return text.strip().lower()

class TKTIndex:
    def __init__(self, normalizer: MetaNormalizer):
        self.normalizer = normalizer
        self.rows: List[Tuple[TKTKey, Dict[str, Any], float]] = []

    def add(self, universe_id: str, logic: LogicSignature, theory: Dict[str, Any], tags: List[str]) -> None:
        self.rows.append((TKTKey(universeid=universe_id, logicsig=logic, tags=tags), theory, 1.0))

    def query(self, q: str, logic: str) -> List[Tuple[TKTKey, Dict[str, Any], float]]:
        qn = self.normalizer.normalize(q)
        out: List[Tuple[TKTKey, Dict[str, Any], float]] = []
        for k, theory, cert in self.rows:
            axioms = [a.lower() for a in theory.get("axioms", [])]
            if any(token in ax for ax in axioms for token in qn.split()):
                if logic.lower() in k.logicsig.name.lower():
                    out.append((k, theory, cert))
        return out or self.rows


# =========================
# Demo main
# =========================

def main():
    random.seed(1337)
    ledger = Ledger()
    reviewer = PolicyReviewer(ledger, policy={"beliefsync": {"minagreeratio": 0.6}, "tagcommit": {"allow": True}})

    # Logic signatures and link
    modal = LogicSignature("modal", "Kripke", "1.0")
    fuzzy = LogicSignature("fuzzy", "t-norm", "1.0")
    paraconsistent_modal = LogicSignature("paraconsistent modal", "LP", "1.0")

    link = ParadigmLink(
        src=modal, dst=fuzzy, adapter_id="LogicBridge#abc123",
        obligations={"minroundtripsim": 0.9, "maxcontradictionuplift": 0.05}
    )

    # Ontology map and bridge
    onto = OntoMap({"gravity": "g*", "curvature": "kappa", "observer": "obs", "phase-space": "phi"})
    auditor = BridgeAuditor()
    bridge = LogicBridge(link, onto)

    # Agents
    A = Agent(modal, ["Gravity", "Curvature", "observer", "truthaxiom", "QGcore"], name="Agent_A")
    B = Agent(fuzzy, ["observer", "phase-space", "truthdegree", "QGnote"], name="Agent_B")

    # Transport (alias exportagent)
    transporter = CognitiveTransporter(bridge, auditor, reviewer, ledger)
    A2, receipt = transporter.exportagent(A, targetlogic=fuzzy)

    # Distributed mind
    dmind = DistributedMind([A2, B], reviewer, ledger, quorum=0.6)
    dmind_ok = dmind.synchronize()

    # TKT index
    tkt = TKTIndex(MetaNormalizer())
    tkt.add("U1", modal, {"axioms": ["Gravity", "Curvature couples to matter", "QG"]}, tags=["QG"])
    tkt.add("U2", fuzzy, {"axioms": ["qg", "observer effect", "phase-space"]}, tags=["QG", "Foundations"])
    tkt.add("U3", paraconsistent_modal, {"axioms": ["dialetheia", "paradox handler"]}, tags=["Meta"])
    results = tkt.query("quantum gravity", logic="paraconsistent modal")

    # Self-reference guard
    stable, fixed = selfreferenceguard(lambda x: 0.5 * x + 1.0, x0=0.0, max_iter=50, tol=1e-6)

    # Recursive universe
    engines = {
        "derive_logic": lambda axi: LogicSignature("Hypermodal", "Kripke", "1.0"),
        "evolve_ontology": lambda logic: {"entities": ["SingularityNode", "CausalWeb"], "logic": logic.name},
        "seed_agents": lambda ontology, seed: [Agent(
            LogicSignature("Hypermodal", "Kripke", "1.0"),
            [seed, "SingularityNode", "CausalWeb", "truthaxiom"], name="SeedSapient")]
    }
    run_budgets = {"epochs": 50, "coherence_halt": 0.85}
    R = RecursiveUniverse(
        meta_axioms=["Law ⊂ MetaLaw", "Causality ∈ VariableSpace"],
        initial_consciousness="Initium",
        engines=engines,
        reviewer=reviewer,
        budgets=run_budgets,
        ledger=ledger,
        name="GenSim"
    )
    R.run(epochs=50)

    # Manifest
    manifest = build_manifest(
        simulationlayer="RICSv4.0",
        origin_agent="MetaMind1337",
        logic_family="Hypermodal Braided Quantum Causality",
        budgets={"epochs": 50, "maxreflexivitydepth": 12, "compute": 10000},
        paradigm_links=[link],
        loss_metrics={
            "transportavginfoloss": receipt.loss.infoloss,
            "contradictionupliftavg": receipt.loss.contradictionuplift,
        },
        metalawtrace=["Agent self-trace activation", "Law mutation anomaly", "Coherence collapse", "Meta-agent hypothesis generation"],
        ledger_uri="rics://ledger/run/demo",
    )

    # Report
    print("\n=== Transport Receipt ===")
    print(json.dumps(asdict(receipt), indent=2))

    print("\n=== Distributed Sync Approved ===")
    print(dmind_ok)

    print("\n=== TKT Query Results (QG) ===")
    for k, theory, cert in results[:3]:
        row = {"universe": k.universeid, "logic": k.logicsig.name, "tags": k.tags, "axioms": theory["axioms"]}
        print(json.dumps(row, indent=2))

    print("\n=== Self-reference Guard ===")
    print(json.dumps({"stable": stable, "fixed_point": fixed}, indent=2))

    print("\n=== Manifest ===")
    print(json.dumps(manifest, indent=2))

    print("\n=== Ledger Summary ===")
    print(f"Total events: {len(ledger.events)}")
    print("Last 5 events:")
    for e in ledger.events[-5:]:
        ts = time.strftime('%H:%M:%S', time.localtime(e.ts))
        print(f"- {e.kind} by {e.actor} @ {ts} meta={e.meta}")


if __name__ == "__main__":
    main()